# Make your own forcing data from ERA5

Introduction: There are lots of forcing data can be used to drive the model, such as ERA5, NCEP (https://psl.noaa.gov/data/gridded/data.ncep.reanalysis.html), etc. Here, we recommend ERA5 to make your own single point forcing as the NCEP and others are too big to download. The ERA5 data can be obtained in a small scale, which usually light weight to download and store.

## Step 1: Download the required variables from the cdsapi

First, we will use the `cdsapi` package to download the data. If you don't have the package installed, you can install it using the following command:

```
pip install cdsapi

cat <<EOF > ~/.cdsapirc
url: {api-url}
key: {uid}:{api-key}
EOF 
```

[How to get your CDS API?](https://cds.climate.copernicus.eu/how-to-api)

We also can use the interface to download the data. The code below is an example of how to download the data using the interface.

ref: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form

In [1]:
from pyclmuapp import era5_download, era5_to_forcing

In [2]:
# Download ERA5 data for a single month
# if file already exists, it will not be downloaded again
pres, single = era5_download(year=2012, month=8, pressure_level=950, lat=51.5, lon=0.12, outputfolder='./data')
# it will take times to download the data
# including quequing time, and post-processing time, downloading time 
# is usually more than minutes
# here is the example, that we have already downloaded the data, so it will not download again

file exists: ./data/era5_pressure_2012_08_51.5_0.12.nc
file exists: ./data/era5_single_2012_08_51.5_0.12.nc


In [3]:
lat, lon, pressure = 51.5, 0.12, 950
month, year = 8, 2012
# Convert ERA5 data to CLM forcing
forcing = era5_to_forcing(pres=pres, single=single, 
                          lat=lat, lon=lon, pressure=950, 
                          outputfile=f'./data/era5_forcing_{lat}_{lon}_{pressure}_{year}_{str(month).zfill(2)}.nc')

In [4]:
forcing

<xarray.Dataset>
Dimensions:   (time: 744, y: 1, x: 1)
Coordinates:
  * time      (time) datetime64[ns] 2012-08-01 ... 2012-08-31T23:00:00
  * x         (x) int64 1
  * y         (y) int64 1
Data variables:
    PSurf     (time, y, x) float64 9.5e+04 9.5e+04 9.5e+04 ... 9.5e+04 9.5e+04
    Qair      (time, y, x) float32 0.007916 0.008096 ... 0.005529 0.005748
    Wind      (time, y, x) float32 10.63 10.63 10.49 10.83 ... 5.706 5.968 5.934
    Tair      (time, y, x) float32 287.9 287.6 287.3 287.3 ... 282.9 282.8 282.4
    Zbot      (time, y, x) float32 497.4 495.2 492.4 489.6 ... 614.5 612.1 612.5
    SWdown    (time, y, x) float32 3.472e-05 3.472e-05 ... 3.472e-05 3.472e-05
    LWdown    (time, y, x) float32 314.2 313.1 320.9 315.0 ... 348.0 347.9 355.8
    Prectmms  (time, y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-18 10:51:14 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

### how to combine multiple months?

In [5]:
import xarray as xr
era5_list = []
# from 2002 to 2014
years = [2012]
# download data from January to December
months = range(1, 13)
lat, lon, pressure = 51.5, 0.12, 950
outputfile='./data/era5_forcing_{lat}_{lon}_{pressure}_{year}_{month}.nc'
for year in years:
    for month in months:
        pres, single = era5_download(year=year, month=month, pressure_level=pressure,
                                     lat=lat, lon=lon, outputfolder='./data')
        # Convert ERA5 data to CLM forcing
        forcing = era5_to_forcing(pres=pres, single=single, 
                                lat=lat, lon=lon, pressure=pressure,
                                outputfile=outputfile.format(lat=lat, lon=lon, 
                                                             pressure=pressure, year=year, 
                                                             month=str(month).zfill(2)))
        ds = xr.open_dataset(outputfile.format(lat=lat, lon=lon, 
                                               pressure=pressure, year=year, 
                                               month=str(month).zfill(2)))
        era5_list.append(ds)
era5 = xr.concat(era5_list, dim='time').sortby('time')
era5.to_netcdf(f'./data/era5_forcing_{lat}_{lon}_{pressure}.nc')

download: ./data/era5_pressure_2012_01_51.5_0.12.nc


2024-06-18 12:38:29,567 INFO Welcome to the CDS
2024-06-18 12:38:29,568 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-06-18 12:38:29,655 INFO Request is queued
2024-06-18 12:38:32,264 INFO Request is running
2024-06-18 12:41:21,284 INFO Request is completed
2024-06-18 12:41:21,287 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data5/adaptor.mars.internal-1718710870.1279616-13370-2-0b0da626-3704-48f0-800c-240ee9193a07.nc to ./data/era5_pressure_2012_01_51.5_0.12.nc (70.5K)
2024-06-18 12:41:21,610 INFO Download rate 219.2K/s
2024-06-18 12:41:21,697 INFO Welcome to the CDS
2024-06-18 12:41:21,698 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-06-18 12:41:21,755 INFO Request is queued


download: ./data/era5_single_2012_01_51.5_0.12.nc


2024-06-18 12:41:42,835 INFO Request is running
2024-06-18 12:47:40,297 INFO Request is completed
2024-06-18 12:47:40,301 INFO Downloading https://download-0019.copernicus-climate.eu/cache-compute-0019/cache/data6/adaptor.mars.internal-1718711168.8172808-13500-2-567b1ee0-3e87-4b9d-b534-77cc598552a5.nc to ./data/era5_single_2012_01_51.5_0.12.nc (57.2K)
2024-06-18 12:47:40,616 INFO Download rate 181.5K/s
2024-06-18 12:47:40,981 INFO Welcome to the CDS
2024-06-18 12:47:40,981 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-06-18 12:47:41,128 INFO Request is queued


download: ./data/era5_pressure_2012_02_51.5_0.12.nc


...

2024-06-18 13:54:56,251 INFO Request is running
2024-06-18 14:01:13,698 INFO Request is completed
2024-06-18 14:01:13,701 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data7/adaptor.mars.internal-1718715559.3568072-19123-14-0ee1c6e4-f000-4215-8ae6-7cb365ea830c.nc to ./data/era5_single_2012_11_51.5_0.12.nc (55.4K)
2024-06-18 14:01:14,014 INFO Download rate 176.9K/s
2024-06-18 14:01:14,147 INFO Welcome to the CDS
2024-06-18 14:01:14,148 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-06-18 14:01:14,209 INFO Request is queued


download: ./data/era5_pressure_2012_12_51.5_0.12.nc


2024-06-18 14:01:19,108 INFO Request is running
2024-06-18 14:04:05,800 INFO Request is completed
2024-06-18 14:04:05,803 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data7/adaptor.mars.internal-1718715827.6955144-6017-8-373cee8d-d9a0-40a5-ab2e-4b15b429f753.nc to ./data/era5_pressure_2012_12_51.5_0.12.nc (70.5K)
2024-06-18 14:04:06,119 INFO Download rate 223.8K/s
2024-06-18 14:04:06,208 INFO Welcome to the CDS
2024-06-18 14:04:06,209 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-06-18 14:04:06,285 INFO Request is queued


download: ./data/era5_single_2012_12_51.5_0.12.nc


2024-06-18 14:04:11,187 INFO Request is running
2024-06-18 14:08:24,582 INFO Request is completed
2024-06-18 14:08:24,586 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data7/adaptor.mars.internal-1718716096.2338367-20653-11-643c73f4-3caf-442e-bb7d-4fff0534e508.nc to ./data/era5_single_2012_12_51.5_0.12.nc (57.2K)
2024-06-18 14:08:25,084 INFO Download rate 115K/s  


In [6]:
era5

<xarray.Dataset>
Dimensions:   (time: 8784, y: 1, x: 1)
Coordinates:
  * time      (time) datetime64[ns] 2012-01-01 ... 2012-12-31T23:00:00
  * x         (x) int64 1
  * y         (y) int64 1
Data variables:
    PSurf     (time, y, x) float64 9.5e+04 9.5e+04 9.5e+04 ... 9.5e+04 9.5e+04
    Qair      (time, y, x) float32 0.007602 0.0076 ... 0.006078 0.005956
    Wind      (time, y, x) float32 17.46 17.65 16.68 15.61 ... 13.22 13.56 13.36
    Tair      (time, y, x) float32 282.3 282.3 282.3 282.2 ... 280.2 280.2 280.1
    Zbot      (time, y, x) float32 442.4 443.3 443.1 439.8 ... 363.6 362.4 363.8
    SWdown    (time, y, x) float32 1.736e-05 1.736e-05 1.736e-05 ... 0.0 0.0 0.0
    LWdown    (time, y, x) float32 364.3 364.9 366.4 367.3 ... 332.9 336.0 335.0
    Prectmms  (time, y, x) float32 8.402e-05 1.508e-05 ... 2.431e-06 1.389e-06
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-18 11:41:16 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

### use command line to get data

In [7]:
! pyclmuapp --pyclmuapp_mode get_forcing \
    --lat 51.5 --lon 0.12 --pressure 950 \
    --start_year 2012 --end_year 2012 \
    --start_month 1 --end_month 1
# will download and save in the default folder `./era5_forcing/`
# the output file will be `./era5_forcing/era5_forcing_51.5_0.12_950_2012_01.nc`

Namespace(init=False, pwd='/Users/user/Documents/GitHub/pyclmuapp', container_type='docker', input_path=None, output_path=None, log_path=None, scripts_path=None, pyclmuapp_mode='get_forcing', has_container=True, usr_domain=None, usr_forcing=None, usr_surfdata=None, output_prefix='_clm.nc', case_name='usp_case', run_startdate=None, stop_option='ndays', stop_n='1', run_type='coldstart', run_refcase='None', run_refdate='None', iflog=True, logfile='pyclmuapp.log', hist_type='GRID', hist_nhtfrq=1, hist_mfilt=1000000000, clean=False, surf_var=None, surf_action=0, forcing_var=None, forcing_action=0, script=None, urbsurf=None, soildata=None, pct_urban=[0, 0, 100.0], lat=51.5, lon=0.12, outputname='surfdata.nc', pressure=950, start_year=2012, end_year=2012, start_month=1, end_month=1)
download: ./era5_data/era5_pressure_2012_01_51.5_0.12.nc
2024-06-18 14:19:59,968 INFO Welcome to the CDS
2024-06-18 14:19:59,968 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysi